# Basic work

### Setup

In [1]:
var moment = require('moment');
var { constants } = require('../utils');
var { advanceTimeAndBlock } = require('../utils/evm');
var { toUnit } = require('../utils/bn');
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm');
var { next, clear } = require('../utils/jupyter');
var { Notebook } = require('../utils/notebook');

clear();

In [2]:
var notebook = new Notebook();

In [3]:
next(async () => {
    await notebook.setup();
});

### Add LP tokens to job

In [4]:
next(async () => {
    await notebook.addLiquidityToJob(toUnit(1));
});

### Work, work, work
Sleep 72 hours, then start working every 72 hours, for a total of 4 periods

In [5]:
next(async () => {
    const timeToWork = notebook.rewardPeriod * 4;
    const startedToWorkAt = await getLatestBlockTimestamp();
    const maxRestHours = 72;
    
    await notebook.recordCredits();
    await advanceTimeAndBlock(moment.duration(maxRestHours, 'hours').as('seconds'));
    await notebook.recordCredits();
    
    while ((await getLatestBlockTimestamp()) - startedToWorkAt < timeToWork) {
        const workRest = moment.duration(maxRestHours, 'hours').as('seconds');
        
        console.log(`Working`);
        await notebook.job.connect(notebook.keeper).work();
        await notebook.recordCredits();
        
        console.log(`Sleeping ${moment.duration(workRest, 'seconds').as('hours')} hours`);
        await advanceTimeAndBlock(workRest);
        await notebook.recordCredits();
    }
});

In [6]:
next(async () => {
    await notebook.draw();
});

Working
Sleeping 72 hours
Working
Sleeping 72 hours
Working
Sleeping 72 hours
Working
Sleeping 72 hours
Working
Sleeping 72 hours
Working
Sleeping 72 hours
